In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"
# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.co

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-04-30 17:44:43--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-04-30 17:44:43 (5.97 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession

In [4]:
spark_engine = SparkSession.builder.appName("bigdata_level_1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
# Load Amazon Data into Spark DataFrame

from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark_engine.sparkContext.addFile(url)
amazon_reviews_df= spark_engine.read.csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

In [6]:
# Filter by votes
filter_dataset = amazon_reviews_df.select(["star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])

In [7]:
# Filter for greater than 20 total votes
filter_dataset= filter_dataset.filter(filter_dataset["total_votes"]>20)
filter_dataset.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|           30|         30|   N|                Y|
|          1|            8|         30|   N|                N|
|          5|           35|         37|   N|                Y|
|          5|           19|         22|   N|                Y|
|          5|           27|         28|   N|                Y|
|          4|           25|         26|   N|                Y|
|          5|           25|         27|   N|                Y|
|          5|           26|         28|   N|                Y|
|          5|           48|         49|   N|                N|
|          5|           49|         52|   Y|                N|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [8]:
# # Filter for greater than 50% of helpful votes
helpful_percentage_df = filter_dataset.filter(filter_dataset["helpful_votes"]/filter_dataset["total_votes"]>=0.5)
helpful_percentage_df.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|           30|         30|   N|                Y|
|          5|           35|         37|   N|                Y|
|          5|           19|         22|   N|                Y|
|          5|           27|         28|   N|                Y|
|          4|           25|         26|   N|                Y|
|          5|           25|         27|   N|                Y|
|          5|           26|         28|   N|                Y|
|          5|           48|         49|   N|                N|
|          5|           49|         52|   Y|                N|
|          5|           27|         28|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [9]:
# Descriptive statistics for paid reviews
vine_df = helpful_percentage_df.filter(helpful_percentage_df["vine"]== 'Y')
vine_df.describe().show(10)

+-------+------------------+------------------+------------------+----+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes|vine|verified_purchase|
+-------+------------------+------------------+------------------+----+-----------------+
|  count|                43|                43|                43|  43|               43|
|   mean|3.9069767441860463| 45.02325581395349| 50.51162790697674|null|             null|
| stddev|1.0870220019758676|53.599257445146534|56.920735234778675|null|             null|
|    min|                 1|                13|                21|   Y|                N|
|    max|                 5|               349|               370|   Y|                N|
+-------+------------------+------------------+------------------+----+-----------------+



In [11]:
# Descriptive statistics for unpaid reviews
vine_df_n = helpful_percentage_df.filter(helpful_percentage_df["vine"]== 'N')
vine_df_n.describe().show(10)

+-------+------------------+-----------------+-----------------+----+-----------------+
|summary|       star_rating|    helpful_votes|      total_votes|vine|verified_purchase|
+-------+------------------+-----------------+-----------------+----+-----------------+
|  count|              7750|             7750|             7750|7750|             7750|
|   mean|3.8033548387096774|44.35006451612903|49.28812903225806|null|             null|
| stddev|1.5225882483738482|76.50421718944474|82.83273224719669|null|             null|
|    min|                 1|               11|               21|   N|                N|
|    max|                 5|             4004|             4249|   N|                Y|
+-------+------------------+-----------------+-----------------+----+-----------------+

